In [ ]:
from flask import request
from flask import jsonify
from flask import Flask
from flask_cors import *

import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
tf.disable_v2_behavior()
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image


from utils import label_map_util
from utils import visualization_utils as vis_util

from collections import Counter

import warnings
warnings.filterwarnings("ignore")

app = Flask(__name__) 
CORS(app)
app.config['CORS_HEADERS']='Content-Type'


# What model to download.
MODEL_NAME = 'damagic_graph'
# MODEL_FILE = MODEL_NAME + '.tar.gz'
# DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = r"C:\Users\Prem\Desktop\projects\damagic\object_detection\object-detection.pbtxt"

NUM_CLASSES = 4


# ## Download Model

# In[5]:

# opener = urllib.request.URLopener()
# opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
# tar_file = tarfile.open(MODEL_FILE)
# for file in tar_file.getmembers():
#     file_name = os.path.basename(file.name)
#     if 'frozen_inference_graph.pb' in file_name:
#         tar_file.extract(file, os.getcwd())


# ## Load a (frozen) Tensorflow model into memory.
def model_pred():
            # ## Download Model

    # In[5]:

    # opener = urllib.request.URLopener()
    # opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
    # tar_file = tarfile.open(MODEL_FILE)
    # for file in tar_file.getmembers():
    #     file_name = os.path.basename(file.name)
    #     if 'frozen_inference_graph.pb' in file_name:
    #         tar_file.extract(file, os.getcwd())


    # ## Load a (frozen) Tensorflow model into memory.

    # In[6]:
    detection_graph = tf.Graph()
    with detection_graph.as_default():
        od_graph_def = tf.compat.v1.GraphDef()
        with tf.compat.v2.io.gfile.GFile(r"C:\Users\Prem\Desktop\projects\damagic\object_detection\damagic_graph\frozen_inference_graph.pb", 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')


    # ## Loading label map
    # Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

    # In[7]:

    label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
    categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)


    # ## Helper code

    # In[8]:

    def load_image_into_numpy_array(image):
        (im_width, im_height) = image.size
        return np.array(image.getdata()).reshape(
            (im_height, im_width, 3)).astype(np.uint8)


    # # Detection

    # In[9]:

    # For the sake of simplicity we will use only 2 images:
    # image1.jpg
    # image2.jpg
    # If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
    PATH_TO_TEST_IMAGES_DIR = r"C:\Users\Prem\Desktop\projects\damagic\object_detection\test_images"
    # TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpeg'.format(i)) for i in range(1, 21) ]  # change this value if you want to add more pictures to test
    # for item in os.listdir(PATH_TO_TEST_IMAGES_DIR):
    #     if not item.startswith('.') and os.path.isfile(os.path.join(PATH_TO_TEST_IMAGES_DIR, item)):
    #         print(item)

    TEST_IMAGE_PATHS=[]

    for item in os.listdir(PATH_TO_TEST_IMAGES_DIR):
        if not item.startswith('.') and os.path.isfile(os.path.join(PATH_TO_TEST_IMAGES_DIR, item)):
            TEST_IMAGE_PATHS.append(os.path.join(PATH_TO_TEST_IMAGES_DIR,item))


    # Size, in inches, of the output images.
    IMAGE_SIZE = (20, 20)


    # In[10]:
    with detection_graph.as_default():
        with tf.compat.v1.Session(graph=detection_graph) as sess:
            i = 0   # add variable for a janky fix
            for image_path in TEST_IMAGE_PATHS:
                image = Image.open(image_path)
                # the array based representation of the image will be used later in order to prepare the
                # result image with boxes and labels on it.
                image_np = load_image_into_numpy_array(image)
                # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
                image_np_expanded = np.expand_dims(image_np, axis=0)
                image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
                # Each box represents a part of the image where a particular object was detected.
                boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
                # Each score represent how level of confidence for each of the objects.
                # Score is shown on the result image, together with the class label.
                scores = detection_graph.get_tensor_by_name('detection_scores:0')
                classes = detection_graph.get_tensor_by_name('detection_classes:0')
                num_detections = detection_graph.get_tensor_by_name('num_detections:0')
                # Actual detection.
                (boxes, scores, classes, num_detections) = sess.run(
                    [boxes, scores, classes, num_detections],
                    feed_dict={image_tensor: image_np_expanded})
                
                # Visualization of the results of a detection.
                new_img = vis_util.visualize_boxes_and_labels_on_image_array(
                    image_np,
                    np.squeeze(boxes),
                    np.squeeze(classes).astype(np.int32),
                    np.squeeze(scores),
                    category_index,
                    use_normalized_coordinates=True,
                    line_thickness=4)
                
#                 j = np.squeeze(classes).astype(np.int32)
#                 print(Counter(j).keys())
#                 print(Counter(j).values())
                plt.figure(figsize=IMAGE_SIZE)
                plt.axis('off')
                plt.imshow(image_np)
                plt.show()    # when running on local PC, matplotlib is not set for graphical display so instead we
                # can save the outputs to a folder I named outputs (make sure to add this folder into object_detection)
                plt.savefig(r'C:\Users\Prem\Downloads\mainproject-main 2\mainproject-main\Mernauthentication\loginregreactjsmui1\src\output\detection_output.png'.format(i), bbox_inches='tight',pad_inches = 0,dpi=100)
                i = i+1
                return ([category_index.get(i)['name'] for i in classes[0]][0])


@app.route("/predict", methods=["POST","GET"])
@cross_origin()
def predict():
    damage_class=model_pred()
    response = {"response":damage_class}
    return jsonify(response)

if __name__ == '__main__' :
    app.run()
    


C:\Users\Prem\anaconda3\envs\damagic\lib\site-packages\requests\__init__.py:104: RequestsDependencyWarning: urllib3 (1.25.8) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  RequestsDependencyWarning)



Instructions for updating:
non-resource variables are not supported in the long term
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [02/Dec/2022 02:29:29] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 02:29:34] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 02:29:39] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 02:29:44] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 02:29:50] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 02:29:55] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 02:30:00] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 02:30:05] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 02:30:10] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 02:30:17] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 02:30:18] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 02:30:19] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 02:30:22] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 02:30:25] "GET /predict HTTP/1.1" 200 -
127.0

127.0.0.1 - - [02/Dec/2022 02:34:45] "GET /predict HTTP/1.1" 500 -
127.0.0.1 - - [02/Dec/2022 02:34:45] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 02:34:48] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 02:34:49] "GET /predict HTTP/1.1" 200 -
[2022-12-02 02:34:55,791] ERROR in app: Exception on /predict [GET]
Traceback (most recent call last):
  File "C:\Users\Prem\anaconda3\envs\damagic\lib\site-packages\flask\app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Prem\anaconda3\envs\damagic\lib\site-packages\flask\app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Prem\anaconda3\envs\damagic\lib\site-packages\flask_cors\extension.py", line 165, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "C:\Users\Prem\anaconda3\envs\damagic\lib\site-packages\flask\app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_reques

127.0.0.1 - - [02/Dec/2022 02:38:39] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 02:38:43] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 02:38:44] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 02:38:46] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 02:38:50] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 02:38:56] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 02:38:59] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 02:39:03] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 02:39:07] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 02:39:08] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 02:39:13] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 02:39:16] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 02:39:19] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 02:39:25] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 02:39:30] "GET /predict HTTP/1.1" 2

...

127.0.0.1 - - [02/Dec/2022 02:57:44] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 02:57:49] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 02:57:55] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 02:58:00] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 02:58:05] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 02:58:10] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 02:58:16] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 02:58:34] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 02:58:40] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 02:58:45] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 02:58:50] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 02:58:55] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 02:59:00] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 02:59:04] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 02:59:10] "GET /predict HTTP/1.1" 2

127.0.0.1 - - [02/Dec/2022 03:17:21] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 03:17:26] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 03:17:31] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 03:17:36] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 03:17:42] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 03:17:47] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 03:17:52] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 03:17:58] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 03:18:03] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 03:18:08] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 03:18:14] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 03:18:19] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 03:18:24] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 03:18:28] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 03:18:33] "GET /predict HTTP/1.1" 2

127.0.0.1 - - [02/Dec/2022 03:28:07] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 03:28:12] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 03:28:18] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 03:28:23] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 03:28:28] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 03:28:33] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 03:28:38] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 03:28:44] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 03:28:49] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 03:28:55] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 03:29:00] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 03:29:05] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 03:29:11] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 03:29:16] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 03:29:21] "GET /predict HTTP/1.1" 2

127.0.0.1 - - [02/Dec/2022 03:53:30] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 03:53:36] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 03:53:42] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 03:53:47] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 03:53:53] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 03:53:59] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 03:54:04] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 03:54:09] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 03:54:15] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 03:54:20] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 03:54:26] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 03:54:31] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 03:54:37] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 03:54:43] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 03:54:48] "GET /predict HTTP/1.1" 2

127.0.0.1 - - [02/Dec/2022 13:01:22] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 13:01:30] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 13:09:37] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 13:53:01] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 13:54:28] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 15:19:16] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 15:38:18] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 15:40:21] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 15:41:17] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 15:41:29] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 15:43:28] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 15:43:39] "GET /predict HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2022 16:11:00] "GET /predict HTTP/1.1" 200 -
